# Relative velocities in DustPy

In [ ]:
import sys

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import cmocean

import dustpy as dp
import dustpy.constants as c

if '..' not in sys.path:
    sys.path.append('..')

import helper

In [ ]:
import dill
from pathlib import Path

In [ ]:
data_dir = '/Users/birnstiel/DATA/araa_data/lowerres_dipsy6/data_nobump/'

In [ ]:
with open(Path(data_dir) / 'frame.dmp', 'rb') as fid:
    s = dill.load(fid)

In [ ]:
reader = dp.hdf5writer()
reader.datadir = data_dir
data = reader.read.output(1)

In [ ]:
import simframe

In [ ]:
def copy(s, data, indent=0):
    "for every field in `s`, this will copy the contents of `data` into `s`."
    ind = indent * ' '
    for key, value in s.__dict__.items():
        if key.startswith('_'):
            continue
            
        if isinstance(value, simframe.frame.Field):
            print(f'{ind}setting field: {key}')
            if hasattr(data, key):
                getattr(s, key)[...] = value[...]
            else:
                print(f'{ind}{key} not in {type(data)}')
                
        elif isinstance(value, simframe.frame.Group):
            print(f'Group: {key}')
            
            if hasattr(data, key):
                copy(value, getattr(data, key), indent=indent + 2)
            else:
                print(f'{ind}{key} not in {type(data)}')
        else:
            raise ValueError('not implemented')

In [ ]:
copy(s, data)

In [ ]:
s.update()

Check Temperature and Surface density values

In [ ]:
np.interp(c.au, s.grid.r, s.gas.T), np.interp(c.au, s.grid.r, s.gas.Sigma)

In [ ]:
keys = {
    'rad': 'Radial Motion',
    'azi': 'Azimuthal Drift',
    'vert': 'Vertical Settling',
    'brown': 'Brownian Motion',
    'turb': 'Turbulent Motion',
    'tot': 'Total'}

In [ ]:
ir = np.abs(s.grid.r - c.au).argmin()
a = s.dust.a[ir]

f, axs = plt.subplots(2, 3, figsize=(12, 8), sharex=True, sharey=True, gridspec_kw={'hspace':0.1, 'wspace':0.1})

vmin = 1e-2
vmax = 1e4
norm = LogNorm(vmin=vmin, vmax=vmax, clip=False)
levels = np.geomspace(vmin, vmax, 3 * int(np.log10(vmax) - np.log10(vmin)) + 1)

white_lines = [100, 1000]

for i, (key, lab) in enumerate(keys.items()):
    ax = axs.ravel()[i]
    
    vrel = getattr(s.dust.v.rel, key)[ir, ...]
    cc = ax.contourf(a, a, vrel + 1e-100, levels, cmap='cmo.algae', norm=norm, extend='both')
    
    # style each plot
    ax.set_aspect('equal')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_title(keys[key])
    ax.minorticks_off()
    
    # add white contour lines
    wc = ax.contour(a, a, vrel + 1e-100, white_lines, colors='w', linestyles=['--', '-'])
    if i == 0:
        manual = [(1e-3, 1e0), (1e-3, 1e2)]
    else:
        manual = False
    ax.clabel(wc, fmt=lambda val: f'{val/100:.0f} m/s', manual=manual)


# color bar

pos1, pos0 = [ax.get_position() for ax in axs.ravel()[[2, -1]]]
w = pos0.width / 10
cax = f.add_axes([pos0.x1 + w, pos0.y0, w, pos1.y1 - pos0.y0])
cb = f.colorbar(cc, cax=cax)
cb.set_label('relative velocity [cm / s]')
cb.set_ticks(10.**np.arange(np.log10(vmin), np.log10(vmax)+1))


# axes ticks and labels

for ax in axs[1,:]:
    ax.set_xlabel('particle size [cm]')
for ax in axs[:, 0]:
    ax.set_ylabel('particle size [cm]')
    
for ax in axs.ravel():
    ax.set_xticks(10.**np.arange(-5, 4))
    ax.set_yticks(10.**np.arange(-5, 4))
    ax.set_xlim(a[[0,-1]])
    ax.set_ylim(a[[0,-1]])

In [ ]:
f.savefig(helper.output_dir / 'BirnstielFig5.pdf', transparent=True, bbox_inches='tight')

## Check some values

In [ ]:
for key in keys.keys():
    val = getattr(s.dust.v.rel, key)
    print(f'{key+":":6} {np.nanmax(val[ir, :, :])/100:>5.2f} m/s')